In [1]:
import torch
import random
import torchvision
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import MultiStepLR
from sklearn.model_selection import train_test_split

from utils.util import get_image_list, ImageDataset, dataset_list, PathDataset
from avalanche.benchmarks.datasets import CelebA

from avalanche.benchmarks.utils import AvalancheTensorDataset
from avalanche.training.supervised import GEM
from avalanche.logging import InteractiveLogger, WandBLogger
from avalanche.benchmarks.generators import nc_benchmark, paths_benchmark
from avalanche.benchmarks.utils import AvalancheDataset, AvalancheDatasetType
from avalanche.benchmarks.utils.dataset_definitions import IDatasetWithTargets
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.plugins.lr_scheduling import LRSchedulerPlugin
from avalanche.evaluation.metrics import ExperienceAccuracy, ExperienceLoss, ExperienceForgetting, ExperienceCPUUsage, ExperienceMaxGPU, ExperienceMaxRAM, ExperienceTime, EpochAccuracy
from avalanche.benchmarks.generators import (
    filelist_scenario, dataset_scenario, tensor_scenario, paths_scenario
)

c:\Users\LAB\Anaconda3\envs\lab\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\LAB\Anaconda3\envs\lab\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\LAB\Anaconda3\envs\lab\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\LAB\Anaconda3\envs\lab\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\Users\LAB\Anaconda3\envs\lab\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
# torch.cuda.manual_seed_all(seed) # if use multi-GPU
cudnn.deterministic = True  # 연산 처리 속도 감소 -> 모델과 코드를 배포해야 하는 연구 후반 단계에 사용
cudnn.benchmark = False
# torch.set_printoptions(sci_mode=False)

In [3]:
transform = transforms.Compose([
    # transforms.Resize(224),
    transforms.ToTensor()
    ])

In [4]:
data_path = "../Data/CASIA-WebFace/"

dataset = get_image_list(data_path)
x_train, x_test, y_train, y_test = train_test_split(dataset[0], dataset[1], test_size=0.2, shuffle=True, random_state=seed, stratify=dataset[1])

train_set = dataset_list(x_train, y_train)
test_set = dataset_list(x_test, y_test)

# train_set = ImageDataset(x_train, y_train)
# test_set = ImageDataset(x_test, y_test)
# train_dataset = TensorDataset(x_train, y_train)
# test_dataset = TensorDataset(x_test, y_test)
# train_dataset = AvalancheTensorDataset(x_train, y_train)
# test_dataset = AvalancheTensorDataset(x_test, y_test)
# train_set = tensor_scenario(x_train, y_train, task_labels=0)
# test_set = tensor_scenario(x_test, y_test)

print("train : ", len(y_train))
print("test : ", len(y_test))
# print("target : ", train_set.traget)

# # check avalanche dataset
# x,y = train_set[0]
# print("avalanche dataset x : ", type(x))
# print("avalanche dataset y : ", type(y))

## Data imbalance
# from collections import Counter
# cnt = Counter(dataset[1])
# print("Target : ", cnt.items())

train :  395531
test :  98883


In [5]:
transforms_group = dict(
       train=(
       transforms.Compose(
              [
              transforms.ToTensor(),
              ]
       ),
       None,
       ),
       eval=(
       transforms.Compose(
              [
              transforms.ToTensor(),
              ]
       ),
       None,
       )
)

# train_set = AvalancheDataset(trian_loader)
# test_set = AvalancheDataset(test_loader)

In [6]:
# task_lable = list(range(10575))
task_lable = [0,1,2,3,4]

# train_set = PathDataset(train_set, transforms_group, "train", task_lable)
# test_set = PathDataset(test_set, transforms_group, "eval", task_lable)
train_set = PathDataset(train_set, transforms_group, "train")
test_set = PathDataset(test_set, transforms_group, "eval")

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

num_class = 10575
incremental = 5
# task_lable = [0,1,2,3,4]

# scenario = paths_benchmark(train_lists_of_files=train_set,
#                            test_lists_of_files=test_set,
#                            task_labels=task_lable,
#                            complete_test_set_only=True,
#                            train_transform=transform,
#                            eval_transform=transform,
#                            dataset_type=AvalancheDatasetType.CLASSIFICATION
# )

scenario = nc_benchmark(train_dataset=train_set,
                        test_dataset=test_set,
                        n_experiences=incremental,
                        task_labels=True,
                        seed=seed,
                        shuffle=False
                        )

model = torchvision.models.resnet18(pretrained=False, num_classes=num_class)
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=1e-1)
criterion = torch.nn.CrossEntropyLoss()

In [8]:
interactive_logger = InteractiveLogger()
wandb_logger = WandBLogger(project_name="Face Recognition", run_name="GEM-CASIA")
eval_plugin = EvaluationPlugin(
    EpochAccuracy(),
    ExperienceAccuracy(),
    ExperienceLoss(),
    ExperienceForgetting(),
    ExperienceCPUUsage(),
    ExperienceMaxGPU(gpu_id=0),
    ExperienceMaxRAM(),
    ExperienceTime(),
    loggers=[interactive_logger, wandb_logger])

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hopo55. Use `wandb login --relogin` to force relogin


c:\Users\LAB\Anaconda3\envs\lab\lib\site-packages\avalanche\training\plugins\evaluation.py:81: UserWarning: No benchmark provided to the evaluation plugin. Metrics may be computed on inconsistent portion of streams, use at your own risk.
  warnings.warn(


## GEM

In [25]:
train_batch = 256
eval_batch = 100
epoch = 10

strategies = GEM(model, optimizer, criterion, patterns_per_exp=256, memory_strength=0.5, train_epochs=epoch, device=device, train_mb_size=train_batch, eval_mb_size=eval_batch, evaluator=eval_plugin)

In [26]:
for i, exp in enumerate(scenario.train_stream):
    eval_exps = [e for e in scenario.test_stream][: i + 1]
    strategies.train(exp)
    strategies.eval(eval_exps)

-- >> Start of training phase << --


RuntimeError: CUDA out of memory. Tried to allocate 382.00 MiB (GPU 0; 8.00 GiB total capacity; 7.22 GiB already allocated; 0 bytes free; 7.28 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF